## Sprintの目的
>公開されている実装を動かす際のノウハウを得る
>論文で読んだ手法を動かしてみる

## Faster R-CNN

>Faster R-CNN[1]の実装を動かします。
>
>[1]Ren, S., He, K., Girshick, R., Sun, J.: Faster r-cnn: Towards real-time object detection with region proposal networks. In: Advances in neural information processing systems. (2015) 91–99
>
>https://arxiv.org/pdf/1506.01497.pdf
>
>以下のものを使用してください。Kerasを使用した実装です。
>
>duckrabbits/ObjectDetection at master

## 【問題1】学習と推定
>READMEを参考に上記実装を動かしてください。



In [4]:
# 動作可能なVersionに合わせる TensorFlow1.14、Keras 2.2.4
# Keras 2.1.5
# tensorflow 1.6.0
!pip install tensorflow==1.14

     |████████████████████████████████| 109.2MB 38kB/s 
     |████████████████████████████████| 51kB 9.0MB/s 
     |████████████████████████████████| 3.2MB 52.0MB/s 
     |████████████████████████████████| 491kB 46.8MB/s 
  Found existing installation: tensorboard 2.4.0
    Uninstalling tensorboard-2.4.0:
      Successfully uninstalled tensorboard-2.4.0
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: tensorflow 2.4.0
    Uninstalling tensorflow-2.4.0:
      Successfully uninstalled tensorflow-2.4.0


In [5]:
import tensorflow

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [6]:
tensorflow.__version__

'1.14.0'

In [7]:
# 動作可能なVersionに合わせる TensorFlow1.14、Keras 2.2.4
# Keras 2.1.5
# tensorflow 1.6.0
!pip install keras==2.2.4

     |████████████████████████████████| 317kB 9.0MB/s 
  Found existing installation: Keras 2.4.3
    Uninstalling Keras-2.4.3:
      Successfully uninstalled Keras-2.4.3


In [8]:
# インストールするとKaggleのファイルが使える
!pip install kaggle

In [9]:
# 自分のマイドライブにマウントする

from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [10]:
mkdir .kaggle

In [11]:
# -aオプションを使うと、ファイル名の先頭にピリオドがあるファイルも表示する。
# 先頭にピリオドがあるファイルはLinuxのシステムと結びつき、普段簡単には編集されたくない設定ファイルなど
!ls -a

.  ..  .config	drive  .kaggle	sample_data


In [13]:
# https://www.kaggle.com/　へアクセスし、ログイン後に右上のプロフィール画像をクリック、さらに"My Account"をクリックする
# このサイトの「API」項目にある"Create New API Token"をクリックすると、kaggle.jsonファイルが自動的にダウンロードされる
# ローカルで、kaggle.json（ダウンロードフォルダにあるはず）をエディターで開く。
# このセルの以下のコードにある token = {'username':'***','key':'***'} における「***」部分を、
# ダウンロードしたkaggle.jsonを参照して書き換え、このセルを実行する

import json

token = {"username":"tomoakiishii","key":"a7d66c5914c0c857351f0a4dfb2499b0"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

In [14]:
# chmodはファイル or ディレクトリに対する権限を設定するコマンド。
# ユーザー区分ごとに権限を設定できる。「所有者権限、グループ権限、その他のユーザーの権限」0~7（r, w, x）
!chmod 600 /content/.kaggle/kaggle.json

In [15]:
pwd

'/content'

In [16]:
ls -a .kaggle

./  ../  kaggle.json


In [17]:
mkdir　/root/.kaggle/

In [18]:
# -R ディレクトリを再帰的にコピーする
!cp -R /content/.kaggle/kaggle.json /root/.kaggle/

In [19]:
!ls -a /root/.kaggle/

.  ..  kaggle.json


In [20]:
# kaggle APIコマンドが使えるかどうかを確認


!kaggle competitions list

ref                                            deadline             category            reward  teamCount  userHasEntered  
---------------------------------------------  -------------------  ---------------  ---------  ---------  --------------  
contradictory-my-dear-watson                   2030-07-01 23:59:00  Getting Started     Prizes         89           False  
gan-getting-started                            2030-07-01 23:59:00  Getting Started     Prizes        187           False  
tpu-getting-started                            2030-06-03 23:59:00  Getting Started  Knowledge        286           False  
digit-recognizer                               2030-01-01 00:00:00  Getting Started  Knowledge       2166           False  
titanic                                        2030-01-01 00:00:00  Getting Started  Knowledge      17070           False  
house-prices-advanced-regression-techniques    2030-01-01 00:00:00  Getting Started  Knowledge       4625            True  
connectx

In [21]:
# カレントディレクトリの変更

import os

os.chdir('/content/drive/MyDrive/')

# カレントディレクトリの取得
print(os.getcwd())

/content/drive/MyDrive


In [22]:
# simpsonデータセットのAPIコマンドでデータセットをダウンロードする
# APIコマンドはこちらのサイトを参照。https://www.kaggle.com/alexattia/the-simpsons-characters-dataset

!kaggle datasets download -d alexattia/the-simpsons-characters-dataset --force

100% 1.08G/1.08G [00:28<00:00, 88.4MB/s]
100% 1.08G/1.08G [00:28<00:00, 40.1MB/s]


In [23]:
# zipファイルの展開。-dで展開時にディレクトリを指定する。
!unzip /content/drive/'MyDrive'/the-simpsons-characters-dataset.zip  -d Simpsons

ストリーミング出力は最後の 5000 行に切り捨てられました。
  inflating: Simpsons/simpsons_dataset/simpsons_dataset/moe_szyslak/pic_1270.jpg  
  inflating: Simpsons/simpsons_dataset/simpsons_dataset/moe_szyslak/pic_1271.jpg  
  inflating: Simpsons/simpsons_dataset/simpsons_dataset/moe_szyslak/pic_1272.jpg  
  inflating: Simpsons/simpsons_dataset/simpsons_dataset/moe_szyslak/pic_1273.jpg  
  inflating: Simpsons/simpsons_dataset/simpsons_dataset/moe_szyslak/pic_1274.jpg  
  inflating: Simpsons/simpsons_dataset/simpsons_dataset/moe_szyslak/pic_1275.jpg  
  inflating: Simpsons/simpsons_dataset/simpsons_dataset/moe_szyslak/pic_1276.jpg  
  inflating: Simpsons/simpsons_dataset/simpsons_dataset/moe_szyslak/pic_1277.jpg  
  inflating: Simpsons/simpsons_dataset/simpsons_dataset/moe_szyslak/pic_1278.jpg  
  inflating: Simpsons/simpsons_dataset/simpsons_dataset/moe_szyslak/pic_1279.jpg  
  inflating: Simpsons/simpsons_dataset/simpsons_dataset/moe_szyslak/pic_1280.jpg  
  inflating: Simpsons/simpsons_dataset/simpsons_dataset

In [25]:
# リポジトリをクローンする(課題のリンク先)
!git clone https://github.com/duckrabbits/ObjectDetection.git

Cloning into 'ObjectDetection'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 49 (delta 4), reused 9 (delta 2), pack-reused 28
Unpacking objects: 100% (49/49), done.


In [26]:
# https://github.com/duckrabbits/ObjectDetection/tree/master
# この後、上述のgithubのreadmeに沿って、simpsons_datasetフォルダをプロジェクトフォルダ配下に格納しましょう。
# testデータなども同様に準備してみよう。

In [30]:
# ファイルやディレクトリを移動
!mv Simpsons/simpsons_dataset ObjectDetection

In [31]:
# annotationファイルはkaggleのではなく、gitにあるファイルをコピーして使う。
!cp -a ./annotation.txt ObjectDetection/annotation.txt

In [33]:
# テスト用の写真が入ったフォルダを作業用フォルダへ移動。
cp -r Simpsons/kaggle_simpson_testset/kaggle_simpson_testset ObjectDetection

In [34]:
ls

 annotation.txt      Simpsons/
 cat.jpg             simpsons_annotation.txt
'Colab Notebooks'/   simpsons_classes.txt
 ObjectDetection/    the-simpsons-characters-dataset.zip


In [39]:
# カレントディレクトリの移動
os.chdir('/content/drive/MyDrive/ObjectDetection')

# カレントディレクトリの取得
print(os.getcwd())

/content/drive/MyDrive/ObjectDetection


In [42]:
!ls

annotation.txt		model	    README.md	      train.py
kaggle_simpson_testset	predict.py  simpsons_dataset


In [46]:
# シンプソンズの写真を学習させる。
!python train.py -p annotation.txt

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

In [48]:
# カレントディレクトリの移動
os.chdir('/content/drive/MyDrive/ObjectDetection')

# カレントディレクトリの取得
print(os.getcwd())

/content/drive/MyDrive/ObjectDetection


In [51]:
# 検証。[]にバウンディングボックスの座標が表示される。表示されない場合は学習が足りていない。
!python predict.py -i './kaggle_simpson_testset' -c './save/train_20201226-150952_config.pickle'

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

## 【問題2】コードリーディング  
>上記実装のコードリーディングを行ってください。
>
>まず、Faster R-CNN[1]において重要だと考えた部分を列挙してください。そして、それに対応するコードを見つけてください。
>
>（例）
>
>- RPNを実現しているコードはどこか
>- RoIプーリングを実現しているコードはどこか
>
>フレームワークには畳み込み層など一般的なものはクラスが用意されていますが、RoIプーリングなど特定の手法限定のものは用意されていません。オリジナルのレイヤーを作成することが可能であり、Kerasであれば以下のページに情報がまとまっています。
>
>オリジナルのKerasレイヤーを作成する - Keras Documentation
>
>《参考》
>
>KerasではVGG16のクラスが用意されているため、簡単に利用ができます。include_top=Falseの引数を与えることで、出力のための全結合層部分が除かれます。weights='imagenet'でImageNetを利用した学習済みモデルも手に入り、転移学習 が行えます。weights='None'とすればランダムな初期化となります。
>
>Applications - Keras Documentation

#### 解答  
- アンカーボックスとAnchorのサイズ
    - data_gen_train = data_generators.get_anchor_gt(train_imgs, classes_count, C, K.image_data_format(), mode='train')
    - data_gen_val = data_generators.get_anchor_gt(val_imgs, classes_count, C, K.image_data_format(), mode='val')
- RPN（region proposal network）  
(画像の各位置でオブジェクトの境界とオブジェクト性スコアを同時に予測する領域検出ネットワーク)
    - model_rpn, model_classifier, model_all = faster_rcnn.get_model(C, classes_count)

- RoIプーリング  
(maxプーリングを使用して有効な関心領域内のフィーチャをH×W（7×7など）の固定空間範囲を持つ小さなフィーチャマップに変換するレイヤー)
    - R = roi_helpers.rpn_to_roi(P_rpn[0], P_rpn[1], C, K.image_data_format(), use_regr=True, overlap_thresh=0.7, max_boxes=300)


## YOLOv3

>シンプソンズのデータセットをFaster R-CNN以外の手法で学習・推定を行います。YOLOv3[2]のKeras実装を使います。
>
>qqwweee/keras-yolo3: A Keras implementation of YOLOv3 (Tensorflow backend)
>
>[2]Jeseph Redmon, Ali Farhadi. YOLOv3: An Incremental Improvement
>
>https://pjreddie.com/media/files/papers/YOLOv3.pdf


## 【問題3】学習済みの重みによる推定
>学習済みの重みを使い推定を行う方法がREADME.mdのQuick Startの項に記載されています。
>
>まずはこの通りにして各自何かしらの画像や動画に対して検出を行ってください。
>
>出力結果を課題の一部として提出してください。

In [27]:
# 動作可能なVersionに合わせる TensorFlow1.14、Keras 2.2.4
# Keras 2.1.5
# tensorflow 1.6.0
!pip install tensorflow==1.6.0

In [28]:
# 動作可能なVersionに合わせる TensorFlow1.14、Keras 2.2.4
# Keras 2.1.5
# tensorflow 1.6.0
!pip install keras==2.1.5

     |████████████████████████████████| 337kB 22.9MB/s 
  Found existing installation: Keras 2.4.3
    Uninstalling Keras-2.4.3:
      Successfully uninstalled Keras-2.4.3


https://pjreddie.com/darknet/yolo/

Easy!

You already have the config file for YOLO in the cfg/ subdirectory.   
You will have to download the pre-trained weight file here (237 MB). Or just run this:

wget https://pjreddie.com/media/files/yolov3.weights

In [29]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
import os

#cd '/content/drive/MyDrive'
os.chdir('/content/drive/MyDrive/')

# カレントディレクトリの取得
print(os.getcwd())

/content/drive/MyDrive


In [32]:
%mkdir yolo
%mkdir yolo/picture

mkdir: cannot create directory ‘yolo’: File exists
mkdir: cannot create directory ‘yolo/picture’: File exists


In [34]:
cd yolo

/content/drive/MyDrive/yolo


In [35]:
!ls

keras-yolo3  picture


In [37]:
# リポジトリをクローンする(課題のリンク先)
!git clone https://github.com/qqwweee/keras-yolo3

fatal: destination path 'keras-yolo3' already exists and is not an empty directory.


In [38]:
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


https://github.com/qqwweee/keras-yolo3  
#### Quick Start
1. Download YOLOv3 weights from YOLO website.
2. Convert the Darknet YOLO model to a Keras model.
3. Run YOLO detection.

In [39]:
cd keras-yolo3

/content/drive/My Drive/yolo/keras-yolo3


In [40]:
# 学習済みの重みを取得する
!wget https://pjreddie.com/media/files/yolov3.weights


--2020-12-26 16:25:49--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights.1’

yolov3.weights.1    100%[===================>] 236.52M  8.91MB/s    in 30s     

2020-12-26 16:26:20 (7.79 MB/s) - ‘yolov3.weights.1’ saved [248007048/248007048]



In [41]:
# 学習済みの重みをコンバート
!python convert.py yolov3.cfg yolov3.weights model_data/yolo.h5

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:520: FutureWarn

In [42]:
pwd

'/content/drive/My Drive/yolo/keras-yolo3'

In [51]:
# テストしたい写真をYOLOのフォルダへ移動。
#!cp -a ../../cat.jpg ../../yolo/picture/cat.jpg
!cp -a ../../cat.jpg cat.jpg

In [52]:
#python yolo_video.py [OPTIONS...] --image, for image detection mode, OR
#python yolo_video.py [video_path] [output_path (optional)]

#!python yolo_video.py --input '../picture/cat.jpg' --output '../picture/cat_yolo.jpg'

# テスト
!python yolo_video.py --image #cat.jpg

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:520: FutureWarn

#### 猫として分類されたことを確認
Input image filename:cat.jpg  
(416, 416, 3)  
Found 1 boxes for img  
cat 1.00 (248, 141) (1027, 617)  
4.680320986000197  

For Tiny YOLOv3, just do in a similar way, just specify model path and anchor path with -

In [53]:
#-model model_file and --anchors anchor_file.

## 【問題4】学習のためのファイルを作成
>新しいデータ（シンプソンズデータセット）を学習します。README.mdのTrainingの項を読み、シンプソンズデータセットを学習するために必要なファイルを作成してください。
>
>アノテーションファイルの形式が問題1の実装とは異なるため変換する必要があります。

## 以下の通り作成

### シンプソンズのannotation.txtの記載形式は以下の通り。  
simpsons_dataset/abraham_grampa_simpson/pic_0000.jpg57,72,52,72,abraham_grampa_simpson  
simpsons_dataset/abraham_grampa_simpson/pic_0001.jpg,80,31,337,354,abraham_grampa_simpson  
simpsons_dataset/abraham_grampa_simpson/pic_0002.jpg,128,48,285,407,abraham_grampa_simpson  
&nbsp;
### クラスファイルとアンカーファイルを作成して以下の形式に変更すること求められる。  
path/to/img1.jpg 50,100,150,200,0 30,50,200,120,3  
path/to/img2.jpg 120,300,250,600,2  

&nbsp;
(実行時に指定する。  
—classes class_file  
--anchors anchor_file)  


&nbsp;
### 以下のファイルを作成。  
- “simpsons_classes.txt”  
- “simpsons_annotation.txt” 

&nbsp;

**`“simpsons_classes.txt”`**  
abraham_grampa_simpson  
agnes_skinner  
apu_nahasapeemapetilon  
barney_gumble  
bart_simpson  
carl_carlson  
charles_montgomery_burns  
chief_wiggum  
cletus_spuckler  
comic_book_guy  
disco_stu  
edna_krabappel  
fat_tony  
gil  
groundskeeper_willie  
homer_simpson  
kent_brockman  
krusty_the_clown  
lenny_leonard  
lionel_hutz  
lisa_simpson  
maggie_simpson  
marge_simpson  
martin_prince  
mayor_quimby  
milhouse_van_houten  
miss_hoover  
moe_szyslak  
ned_flanders  
nelson_muntz  
otto_mann  
patty_bouvier  
principal_skinner  
professor_john_frink  
rainier_wolfcastle  
ralph_wiggum  
selma_bouvier  
sideshow_bob  
sideshow_mel  
simpsons_dataset  
snake_jailbird  
troy_mcclure  
waylon_smithers  

&nbsp;
**`“simpsons_annotation.txt”`**  
../../Simpsons/simpsons_dataset/abraham_grampa_simpson/pic_0000.jpg 57,72,52,72,0  
../../Simpsons/simpsons_dataset/abraham_grampa_simpson/pic_0001.jpg 80,31,337,354,0  
../../Simpsons/simpsons_dataset/abraham_grampa_simpson/pic_0002.jpg 128,48,285,407,0  
../../Simpsons/simpsons_dataset/abraham_grampa_simpson/pic_0003.jpg 72,126,158,275,0  
../../Simpsons/simpsons_dataset/abraham_grampa_simpson/pic_0004.jpg 123,61,294,416,0  

&nbsp;
#### 実行時に指定  
—classes class_file　--anchors anchor_file  
↑コードの書き換えで対応　　





### train.pyの書き換え

def _main():  
&emsp;　annotation_path = **'train.txt'**  
&emsp;　log_dir = 'logs/000/'  
&emsp;　classes_path = **'model_data/voc_classes.txt'**  
&emsp;　anchors_path = 'model_data/yolo_anchors.txt'  
&emsp;　class_names = get_classes(classes_path)  
&emsp;　num_classes = len(class_names)  
&emsp;　anchors = get_anchors(anchors_path)  


def _main():  
&emsp;　annotation_path = **'model_data/simpsons_annotation.txt'**  
&emsp;　log_dir = 'logs/000/'  
&emsp;　classes_path = **'model_data/simpsons_classes.txt'**  
&emsp;　anchors_path = 'model_data/yolo_anchors.txt'  
&emsp;　class_names = get_classes(classes_path)  
&emsp;　num_classes = len(class_names)  
&emsp;　anchors = get_anchors(anchors_path)

In [54]:
pwd

'/content/drive/MyDrive/yolo/keras-yolo3'

In [56]:
# 自作したannotationファイルとclassファイルをマイドライブへアップロードし、model_dataなどのフォルダへ移動
!cp -a '../../simpsons_classes.txt' 'model_data/simpsons_classes.txt'
!cp -a '../../simpsons_annotation.txt' 'model_data/simpsons_annotation.txt'

## 【問題5】学習が行えることの確認
>問題4で作成したファイルを使用して学習してください。実行環境で学習に時間がかかる場合は、学習が行えることを確認するのみで終えて構いません。

#### Training  
Generate your own annotation file and class names file.  
One row for one image;  
Row format: image_file_path box1 box2 ... boxN;  
Box format: x_min,y_min,x_max,y_max,class_id (no space).  
For VOC dataset, try python voc_annotation.py  
Here is an example:  

path/to/img1.jpg 50,100,150,200,0 30,50,200,120,3  
path/to/img2.jpg 120,300,250,600,2  
...  
Make sure you have run python convert.  py -w yolov3.cfg yolov3.weights model_data/yolo_weights.h5  
The file model_data/yolo_weights.h5 is used to load pretrained weights.  

Modify train.py and start training.  
python train.py  
Use your trained weights or checkpoint weights with command line option --model model_file when using yolo_video.py Remember to modify class path or anchor path, with --classes class_file and --anchors anchor_file.



In [57]:
!python convert.py -w yolov3.cfg yolov3.weights model_data/yolo_weights.h5

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:520: FutureWarn

In [ ]:
!python train.py
#!python train.py --classes '../../simpsons_classes.txt' --anchors '../../simpsons_annotation.txt'

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:520: FutureWarn

以上